## Run the STEMMUS_SCOPE model
Steps to run the STEMMUS_SCOPE model, including preprocessing and postprocessing, on Surf super computer Snellius or on a unix system.

In [1]:
import os
from PyStemmusScope import StemmusScope
from PyStemmusScope import save

#### Path to STEMMUS_SCOPE

STEMMUS_SCOPE executable file is located in the **private** repository on GitHub https://github.com/EcoExtreML/STEMMUS_SCOPE. You need to clone the repository locally and specify the path to it in the cell below. Make sure you have right access to the repository. 

#### Update/set config files

In [2]:
# user must provide the correct path
path_to_config_file = "./config_file_template.txt"
path_to_exe_file = "./STEMMUS_SCOPE"

If you run the model locally, you need to first intsall MATLAB Runtime, see instructions in the [readme](https://github.com/EcoExtreML/processing#readme). Then set `LD_LIBRARY_PATH`. To do this, uncomment the cell below and run it:

In [3]:
# # Set LD_LIBRARY_PATH
# matlab_path = !whereis MATLAB
# matlab_path = matlab_path.s.split(": ")[1]
# os.environ['LD_LIBRARY_PATH'] = (
#     f"{matlab_path}/MATLAB_Runtime/v910/runtime/glnxa64:"
#     f"{matlab_path}/MATLAB_Runtime/v910/bin/glnxa64:"
#     f"{matlab_path}/MATLAB_Runtime/v910/sys/os/glnxa64:"
#     f"{matlab_path}/MATLAB_Runtime/v910/extern/bin/glnxa64:"
#     f"{matlab_path}/MATLAB_Runtime/v910/sys/opengl/lib/glnxa64")
# print(os.environ['LD_LIBRARY_PATH'])

/usr/local/MATLAB/MATLAB_Runtime/v910/runtime/glnxa64:/usr/local/MATLAB/MATLAB_Runtime/v910/bin/glnxa64:/usr/local/MATLAB/MATLAB_Runtime/v910/sys/os/glnxa64:/usr/local/MATLAB/MATLAB_Runtime/v910/extern/bin/glnxa64:/usr/local/MATLAB/MATLAB_Runtime/v910/sys/opengl/lib/glnxa64


## Run the model for one forcing file

In [5]:
# create an an instance of the model
model = StemmusScope(config_file=path_to_config_file, exe_file=path_to_exe_file)

# inspect model config template
model.config

{'WorkDir': '/scratch-shared/ecoextreml/stemmus_scope/',
 'SoilPropertyPath': '/projects/0/einf2480/model_parameters/soil_property/',
 'ForcingPath': '/projects/0/einf2480/forcing/plumber2_data/',
 'ForcingFileName': 'FI-Hyy_1996-2014_FLUXNET2015_Met.nc',
 'directional': '/projects/0/einf2480/model_parameters/vegetation_property/directional/',
 'fluspect_parameters': '/projects/0/einf2480/model_parameters/vegetation_property/fluspect_parameters/',
 'leafangles': '/projects/0/einf2480/model_parameters/vegetation_property/leafangles/',
 'radiationdata': '/projects/0/einf2480/model_parameters/vegetation_property/radiationdata/',
 'soil_spectrum': '/projects/0/einf2480/model_parameters/vegetation_property/soil_spectrum/',
 'input_data': '/projects/0/einf2480/model_parameters/vegetation_property/input_data.xlsx',
 'InitialConditionPath': '/projects/0/einf2480/soil_initialcondition/',
 'NumberOfTimeSteps': '5',
 'InputPath': '',
 'OutputPath': ''}

In [6]:
# setup the model
config_path = model.setup(
    ForcingFileName="ZA-Kru_2000-2002_FLUXNET2015_Met.nc",
    NumberOfTimeSteps="10",
)
print(config_path)

/gpfs/scratch1/shared/ecoextreml/stemmus_scope/input/ZA-Kru_2022-08-08-1439/ZA-Kru_2022-08-08-1439_config.txt


In [7]:
# inspect model configs again
model.config

{'WorkDir': '/scratch-shared/ecoextreml/stemmus_scope/',
 'SoilPropertyPath': '/projects/0/einf2480/model_parameters/soil_property/',
 'ForcingPath': '/projects/0/einf2480/forcing/plumber2_data/',
 'ForcingFileName': 'ZA-Kru_2000-2002_FLUXNET2015_Met.nc',
 'directional': '/projects/0/einf2480/model_parameters/vegetation_property/directional/',
 'fluspect_parameters': '/projects/0/einf2480/model_parameters/vegetation_property/fluspect_parameters/',
 'leafangles': '/projects/0/einf2480/model_parameters/vegetation_property/leafangles/',
 'radiationdata': '/projects/0/einf2480/model_parameters/vegetation_property/radiationdata/',
 'soil_spectrum': '/projects/0/einf2480/model_parameters/vegetation_property/soil_spectrum/',
 'input_data': '/projects/0/einf2480/model_parameters/vegetation_property/input_data.xlsx',
 'InitialConditionPath': '/projects/0/einf2480/soil_initialcondition/',
 'NumberOfTimeSteps': '10',
 'InputPath': '/gpfs/scratch1/shared/ecoextreml/stemmus_scope/input/ZA-Kru_2022-

In [8]:
# run the model
result = model.run()
result

b'Opening log file:  /gpfs/scratch1/shared/ecoextreml/stemmus_scope/input/ZA-Kru_2022-08-08-1439//java.log.51183\nReading config from /gpfs/scratch1/shared/ecoextreml/stemmus_scope/input/ZA-Kru_2022-08-08-1439/ZA-Kru_2022-08-08-1439_config.txt\nThe calculations start now\nThe calculations end now\n'

In [9]:
# save output in netcdf format
required_netcdf_variables = "./required_netcdf_variables.csv"
nc_file_name = save.to_netcdf(config_path, required_netcdf_variables)
print(nc_file_name)

/gpfs/scratch1/shared/ecoextreml/stemmus_scope/output/ZA-Kru_2022-08-08-1439/ZA-Kru_2022-08-08-1439_STEMMUS_SCOPE.nc


## Run the model for several forcing files

In [10]:
from pathlib import Path

forcing_filenames_list = [
    "ZA-Kru_2000-2002_FLUXNET2015_Met.nc",
    "AR-SLu_2010-2010_FLUXNET2015_Met.nc",
]

full_run = False
if full_run:
    forcing_filenames_list = [file.name for file in Path(model.config["ForcingPath"]).iterdir()]

for nc_file in forcing_filenames_list:
    # setup the model
    config_path = model.setup(
        ForcingFileName=nc_file,
        NumberOfTimeSteps="10",
    )

    # run the model
    result = model.run()
    print(result)
    
    # save results in a netcdf file
    nc_file_name = save.to_netcdf(config_path, required_netcdf_variables)
    print(nc_file_name)
    

b'Opening log file:  /gpfs/scratch1/shared/ecoextreml/stemmus_scope/input/ZA-Kru_2022-08-08-1442//java.log.63885\nReading config from /gpfs/scratch1/shared/ecoextreml/stemmus_scope/input/ZA-Kru_2022-08-08-1442/ZA-Kru_2022-08-08-1442_config.txt\nThe calculations start now\nThe calculations end now\n'
/gpfs/scratch1/shared/ecoextreml/stemmus_scope/output/ZA-Kru_2022-08-08-1442/ZA-Kru_2022-08-08-1442_STEMMUS_SCOPE.nc
b'Opening log file:  /gpfs/scratch1/shared/ecoextreml/stemmus_scope/input/AR-SLu_2022-08-08-1442//java.log.403\nReading config from /gpfs/scratch1/shared/ecoextreml/stemmus_scope/input/AR-SLu_2022-08-08-1442/AR-SLu_2022-08-08-1442_config.txt\nThe calculations start now\nThe calculations end now\n'
/gpfs/scratch1/shared/ecoextreml/stemmus_scope/output/AR-SLu_2022-08-08-1442/AR-SLu_2022-08-08-1442_STEMMUS_SCOPE.nc
